imports

In [1]:
import config
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import IFrame
import numpy as np
import pickle


In [2]:
with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open("kmeans_model.pkl", "rb") as f:
    kmeans = pickle.load(f)

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                            client_secret= config.client_secret))

In [4]:
df = pd.read_csv("../data/playlist5000_final.csv")
df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,cluster,id,names
0,0.513,0.868,-5.061,0.0410,0.01850,0.000000,0.1210,0.3670,92.009,230467,0,1VU8t2MpJRLG5iFEIxqq7z,Geboren um zu leben
1,0.646,0.359,-10.388,0.1550,0.24700,0.914000,0.0872,0.1540,119.885,360147,6,4YRvb9B8cgvhst11qk8CKn,Sonnentanz
2,0.825,0.652,-3.183,0.0802,0.58100,0.000000,0.0931,0.9310,95.977,233713,2,7qiZfU4dY1lWllzX7mPBI3,Shape of You
3,0.694,0.770,-5.335,0.1490,0.17600,0.000011,0.1180,0.1630,125.905,184560,0,0E9ZjEAyAwOXZ7wJC0PD33,In My Mind
4,0.730,0.769,-5.114,0.0298,0.13800,0.000000,0.0473,0.5070,104.948,228093,0,4OSBTYWVwsQhGLF9NHvIbR,Rolling in the Deep
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4630,0.719,0.580,-6.173,0.0903,0.08180,0.812000,0.1170,0.4900,170.099,131294,6,1Wl0fMRT0YzndDmHZCFvoT,Roadtrip
4631,0.449,0.585,-6.761,0.0268,0.05090,0.000087,0.0833,0.2120,139.631,278719,0,6DGIg7YNSXWz1SAnJ1S8bX,Paradise
4632,0.690,0.723,-9.204,0.0639,0.17400,0.000000,0.2550,0.4870,149.987,157192,4,6WzGciwEdqlkPuMf3bDmX7,Roadtrip
4633,0.626,0.674,-10.002,0.0411,0.00237,0.863000,0.1060,0.0489,124.977,375361,6,2IJZWPp6TnMWHyGqjrz3Ci,Roadtrip - Moritz Hofbauer Remix


In [5]:
song_name = input("Write the song:")
song_name

'hello'

In [6]:
def bring_song(song_name):
    result = sp.search(q=song_name, limit=1, market="DE")
    song_id = result["tracks"]["items"][0]["id"]
    for item in result['tracks']['items']:
        print("The name of song is: '{}' and the id is: {}".format(item['name'],item["id"]))
    view = IFrame(src="https://open.spotify.com/embed/track/"+song_id, width="320", height="80", frameborder="0", allowtransparency="true", allow="encrypted-media")
    return view

chosen_song = bring_song(song_name)
chosen_song

The name of song is: 'Hello' and the id is: 1Yk0cQdMLx5RzzFTYwmuld


In [7]:
def classify_song(song_name):  
        result = sp.search(q=song_name, limit=3, market="DE")
        song_id = result["tracks"]["items"][0]["id"]
        features = sp.audio_features(song_id)
        
        X = pd.DataFrame(features)
        X = X[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
        scaled_X = scaler.transform(X)   
        predicted_cluster = kmeans.predict(scaled_X)[0]
        print("The name of the song is: '{}' and the id is: {}".format(result["tracks"]["items"][0]['name'],song_id))
        print(f"The predicted cluster is: {predicted_cluster}")

classification = classify_song(song_name)
classification


The name of the song is: 'Hello' and the id is: 1Yk0cQdMLx5RzzFTYwmuld
The predicted cluster is: 2


In [33]:
def song_recomender(classification):
        result = sp.search(q=song_name, limit=3, market="DE")
        song_id = result["tracks"]["items"][0]["id"]
        features = sp.audio_features(song_id)
        X = pd.DataFrame(features)
        X = X[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 
                'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
        scaled_X = scaler.transform(X)   
        predicted_cluster = kmeans.predict(scaled_X)[0]
        same_cluster_songs = df.loc[df['cluster'] == predicted_cluster]
        random_sample = same_cluster_songs.sample(n=1)
        recommended_song_name = random_sample['names'].values[0]
        recommended_song_id = random_sample['id'].values[0]
        print(f"Recommended song: '{recommended_song_name}' from the same cluster.")
        
        view = IFrame(src=f"https://open.spotify.com/embed/track/{recommended_song_id}",
                        width="320", height="80",
                        frameborder="0", allowtransparency="true",
                        allow="encrypted-media")
        return view

Recommend = song_recomender(classification)
Recommend

Recommended song: 'Don't You Need Somebody (feat. Enrique Iglesias, R. City, Serayah & Shaggy)' from the same cluster.
